# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

#### Imports & Initialization

In [28]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic

In [29]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

claude_api_key = os.getenv('ANTHROPIC_API_KEY')
if claude_api_key:
    print(f"Claude API Key exists and begins {claude_api_key[:8]}")
else:
    print("Claude API Key not set")
    
openaiMODEL = "gpt-4o-mini"
openai = OpenAI()

claudeMODEL = "claude-3-haiku-20240307"
claude = anthropic.Anthropic(api_key=claude_api_key)

OpenAI API Key exists and begins sk-proj-
Claude API Key exists and begins sk-ant-a


#### Global system message

In [3]:
#the global system message to which we will add later on

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

#### Creating a tool

In [31]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")

    # Add null check
    if destination_city is None:
        print("Warning: destination_city is None")
        return "Unknown"
    
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

Tool get_ticket_price called for None


In [6]:
# There's a particular dictionary structure that's required to describe the function to be used as a tool:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [7]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

#### Translate conversation using claude

In [22]:
# Translation function using Claude
def translate_to_spanish(text):
    try:
        message = claude.messages.create(
            model=claudeMODEL,
            max_tokens=1000,
            temperature=0.7,
            messages=[
                {
                    "role": "user",
                    "content": f"Translate the following text to Spanish. Only provide the translation, no additional text:\n\n{text}"
                }
            ]
        )
        return message.content[0].text
    except Exception as e:
        print(f"Translation error: {e}")
        return f"[Translation error: {text}]"

# def format_conversation_for_translation(history):
#     """Convert conversation history to a readable format for translation"""
#     conversation = []
#     for message in history:
#         if message["role"] == "user":
#             conversation.append(f"Usuario: {message['content']}")
#         elif message["role"] == "assistant":
#             conversation.append(f"Asistente: {message['content']}")
#     return "\n\n".join(conversation)


#### Chat function for gradio, & a tool handler for the chat function

In [19]:
#chat function to use in gradio later on

def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=openaiMODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Translate the entire conversation to Spanish
    spanish_conversation = translate_conversation(history)
    
    return history, image, spanish_conversation

In [32]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [21]:
# fucntion to translate the entire conversation. Entire conversation is stored as a list of message dicts in history, so we can iterate through these message dicts.
def translate_conversation(history):
    """Translate the entire conversation to Spanish"""
    try:
        # Format conversation for translation
        english_text = ""
        for message in history:
            if message["role"] == "user":
                english_text += f"User: {message['content']}\n\n"
            elif message["role"] == "assistant":
                english_text += f"Assistant: {message['content']}\n\n"
        
        if not english_text.strip():
            return ""
            
        # Translate using Claude
        spanish_text = translate_to_spanish(english_text)
        return spanish_text
    except Exception as e:
        print(f"Translation error: {e}")
        return f"Error translating conversation: {str(e)}"

#### Image generator

In [14]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [15]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

#### Gradio Code

In [33]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        with gr.Column(scale=1):
            chatbot = gr.Chatbot(height=500, type="messages", label="English Conversation")
        with gr.Column(scale=1):
            image_output = gr.Image(height=250, label="Destination Image")
            spanish_output = gr.Textbox(
                # height=250, 
                label="Spanish Translation", 
                interactive=False,
                lines=10,
                max_lines=15,
                show_copy_button=True
            )
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    #clear all three outputs (chat, image, and translation) 
    def clear_all():
        return None, None, ""

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output, spanish_output]
    )
    clear.click(clear_all, inputs=None, outputs=[chatbot, image_output, spanish_output], queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Tokyo
Tool get_ticket_price called for New Delhi


<img src="images/book_tickets_to_Berlin.png" alt="Chat bot helps book ticket to Berlin" width="1500">

<img src="images/book_tickets_to_Tokyo.png" alt="Chat bot helps book ticket to Berlin" width="1500">

<img src="images/book_tickets_to_New_Delhi.png" alt="Chat bot helps book ticket to Berlin" width="1500">